<a href="https://colab.research.google.com/github/DmitriySechkin/ds-learning-sb/blob/main/dom_rf_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from tqdm.auto import tqdm

In [3]:
offset_ = 1 # размер шага
limit_ = 100 # количество объектов в выдаче
all_objects_data = []
index = 0

while True:
  url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
  res = requests.get(url)
  data = res.json()
  objects_data = data.get('data').get('list')

  all_objects_data.extend(objects_data)

  print(index)

  if(len(objects_data) == 0):
    print('скачивание данных по объектам завершено')
    break


  offset_ += limit_
  index += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
скачивание данных по объектам завершено


In [4]:
obj_ids = []

for objects_data in tqdm(all_objects_data):
   obj_id = objects_data.get('objId')
   if obj_id:
    obj_ids.append(obj_id)

  0%|          | 0/11653 [00:00<?, ?it/s]

In [5]:
len(obj_ids) == len(all_objects_data)

True

In [6]:
error_request_obj = []

In [7]:
from concurrent.futures import ProcessPoolExecutor

In [9]:

def fetch_data(obj_id):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()
    else:
        error_request_obj.append(obj_id)
        print(f"Ошибка при запросе для obj_id={obj_id}. HTTP статус: {res.status_code}")
        return None


with ProcessPoolExecutor(max_workers=30) as executor:
    results = list(tqdm(executor.map(fetch_data, obj_ids), total=len(obj_ids)))


print(results)

  0%|          | 0/11653 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
results[0]

{'data': {'id': 13398,
  'hobjId': 37165,
  'pdId': 28141,
  'developer': {'devId': 306,
   'devShortCleanNm': '2МЕН ГРУПП',
   'devShortNm': '2МЕН ГРУПП',
   'devFullCleanNm': '2МЕН ГРУПП',
   'problObjCnt': 0,
   'buildObjCnt': 5,
   'comissObjCnt': 0,
   'regRegionDesc': 'Тюменская область',
   'devPhoneNum': '+7(345)279-18-88',
   'devSite': '2mengroup.ru',
   'devEmail': 'info@2mengroup.ru',
   'devInn': '7701651356',
   'devOgrn': '1067746424899',
   'devKpp': '720301001',
   'devLegalAddr': 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5',
   'devFactAddr': 'Тюменская область, город Тюмень, улица Комсомольская 75/5',
   'lastRpdId': 802817,
   'fundGuarantyFlg': 1,
   'devOrgRegRegionCd': 72,
   'devEmplMainFullNm': 'Киселев Сергей Михайлович',
   'developerGroupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
   'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
   'companyGroupId': 5776,
   'objGuarantyEscrowFlg': 0,
   'govFundFlg': 0},
  'region': 72

In [11]:
# если получены не все объекты
if len(error_request_obj) != 0:
    for obj in error_request_obj:
      json = fetch_data(obj)
      if json is not None:
        results.append(json)

filtered_results = [i.get('data') for i in results if i != None]

In [3]:
import pandas as pd

In [42]:
df = pd.json_normalize(filtered_results)
df.head()

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.time,metro.isWalk,metro.colors,complexShortNm,airQualityIndexValue,greenAreaIndexValue,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13398,37165,28141,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,37201,28141,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13383,39024,28105,72,"г Тюмень, ул Профсоюзная, д. 56",4,20,68,2023-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13391,37219,28101,72,"г Тюмень, ул Профсоюзная, д. 56",3,12,44,2023-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10891,6541,23054,35,"г Вологда, ул Медуницинская, д. 11",17,17,225,2023-12-31,Панель,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.shape

(11653, 110)

In [15]:
df.shape[0] == len(obj_ids)

True

In [16]:
name_file = 'building_objects'
df.to_csv(f'{name_file}.csv', index=False)
df.to_excel(f'{name_file}.xlsx', index=False)
df.to_pickle(f'{name_file}.pkl')

In [17]:
from sqlalchemy import create_engine

In [18]:
engine = create_engine('sqlite:///buildings_db.db')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11653 entries, 0 to 11652
Columns: 110 entries, id to developer.orgBankruptMsgDttm
dtypes: bool(2), float64(27), int64(31), object(50)
memory usage: 9.6+ MB


In [ ]:
df1 = df

In [33]:
df1.iloc[:, 23] = df1.iloc[:, 23].astype(str)
df1.iloc[:, 53] = df1.iloc[:, 53].astype(str)
df1.iloc[:, 102] = df1.iloc[:, 102].astype(str)

In [35]:
df1.to_sql('buildings', engine, index=False, if_exists='replace')

11653

In [ ]:
df = pd.read_csv('building_objects.csv')

In [50]:
df['photoRenderDTO'].values.tolist()[0]

[{'objId': 13398,
  'pdId': 28141,
  'rpdId': 802817,
  'devId': 306,
  'ordNum': 1,
  'objRenderPhotoSize': 1325881,
  'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/6C6B7A3A12ECBF4F0CAE418BA061ABE999561FDD',
  'objRenderPhotoDttm': '12-05-2021 16:52',
  'objRenderPhotoHiddenFlg': 0,
  'objRenderPhotoNm': 'IMG_20210512_093624.jpg',
  'objReadyDesc': 'Строится',
  'objBuildTypeShortDesc': 'Жилое',
  'knFlg': 1,
  'loadDttm': '10-12-2023 01:18'},
 {'objId': 13398,
  'pdId': 28141,
  'rpdId': 802817,
  'devId': 306,
  'ordNum': 2,
  'objRenderPhotoSize': 1383679,
  'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/6C6B7A3A342A29C11DCE4370A607229AF564DC02',
  'objRenderPhotoDttm': '12-05-2021 16:52',
  'objRenderPhotoHiddenFlg': 0,
  'objRenderPhotoNm': 'IMG_20210512_0936261.jpg',
  'objReadyDesc': 'Строится',
  'objBuildTypeShortDesc': 'Жилое',
  'knFlg': 1,
  'loadDttm': '10-12-2023 01:18'},
 {'objId': 13398,
  'pdId': 28141,
  'rpdId': 8

In [ ]:
#скачивание картинок
import os
import httplib2
import ast

error_list = []

photos = df['photoRenderDTO'].values.tolist()
print(photos[0])

h = httplib2.Http('.cache')

def download_photos(data):
  data = ast.literal_eval(data)
  for photo in data:
    url = photo.get('objRenderPhotoUrl')
    id = photo.get('objId')
    img_name = photo.get('objRenderPhotoNm').lower()

    if not os.path.isdir(str(id)):
      os.mkdir(str(id))

    response, content = h.request(url)

    if response['status'] != 200:
        error_obj = {}
        error_obj['objRenderPhotoUrl'] = url
        error_obj['objId'] = id
        error_obj['objRenderPhotoNm'] = img_name
        error_list.append(error_obj)

    with open(f"{id}\{img_name}", "wb") as f:
      f.write(content)

with ProcessPoolExecutor(max_workers=30) as executor:
  result = list(tqdm(executor.map(download_photos, photos), total=len(photos)))

print('скачивание картинок завершено')
